<a href="https://colab.research.google.com/github/Jhansipothabattula/Machine_Learning/blob/main/Day52.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building Neural Networks with PyTorch

**Introduction to PyTorch and Its Core Components**

- What is PyTorch?

  - Open-source deep learning framework that provides flexibility and dynamic computation for building and training machine learning models

- Core Components of PyTorch
  
  - Tensors: Multi-dimensional arrays similar to NumPy arrays but with GPU support for accelerated computation

  - Autograd: Automatic differentiation engine that computes gradients for optimization
  
  - torch.nn Module: Provides tools to define and train neural networks with layers, activation functions, and loss functions

**Building a Neural Network in PyTorch**

- Steps

  - Define the Model

    - Use torch.nn.Module to create a neural network with layers and forward propagation
  - Define the Loss Function

    - Use built-in loss functions like Cross-Entropy Loss

  - Define the Optimizer

    - Use optimizers like SGD or Adam for weight updates

**Training, Evaluating, and Saving a Model in PyTorch**

- Training
  
  - Forward pass to compute predictions

  - Compute loss and gradients using backpropagation
  
  - Update weights using an optimizer

- Evaluation

  - Test the model on unseen data and calculate metrics like accuracy

- Saving and Loading

  - Save the model's parameters using torch.save() and load them using torch.load()








**Objective**
- Build, train, evaluate, and save a neural network for MNIST digit classification using PyTorch

In [2]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F

# Define transformation
transformation = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load Datasets
train_dataset = datasets.MNIST(root='./data', train=True, transform=transformation, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transformation, download=True)

# Create Data Loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

print(f"Train Dataset Size: {len(train_dataset)}")
print(f"Test Dataset Size: {len(test_dataset)}")

# Define Model
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.fc1 = nn.Linear(28*28, 128)
    self.fc2 = nn.Linear(128, 64)
    self.fc3 = nn.Linear(64, 10)

  def forward(self, x):
    x = self.flatten(x)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

model = NeuralNetwork()
print(model)

# Define loss function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train(model, train_loader, criterion, optimizer, epochs=5):
  model.train()
  for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in train_loader:
      # Zero gradients
      optimizer.zero_grad()

      # Forward pass
      outputs = model(images)
      loss = criterion(outputs, labels)

      # Backward pass and optimization
      loss.backward()
      optimizer.step()

      running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

train(model, train_loader, criterion, optimizer)

# Evaluation loop
def evaluate(model, test_loader):
  model.eval()
  correct = 0
  total = 0
  with torch.no_grad():
    for images, labels in test_loader:
      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
  accuracy = 100 * correct / total
  print(f"Accuracy on test set: {accuracy:.2f}%")
evaluate(model, test_loader)

# Save the model
torch.save(model.state_dict(), 'mnist_model.pth')

# Reload the model
loaded_model = NeuralNetwork()
loaded_model.load_state_dict(torch.load('mnist_model.pth'))

# Verify loaded model perfomance
evaluate(loaded_model, test_loader)

# Update optimizer with a new learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
train(model, train_loader, criterion, optimizer)
evaluate(model, test_loader)

100%|██████████| 9.91M/9.91M [00:00<00:00, 58.4MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.73MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.5MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 10.2MB/s]


Train Dataset Size: 60000
Test Dataset Size: 10000
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)
Epoch 1, Loss: 0.3537
Epoch 2, Loss: 0.1723
Epoch 3, Loss: 0.1238
Epoch 4, Loss: 0.1040
Epoch 5, Loss: 0.0910
Accuracy on test set: 96.77%
Accuracy on test set: 96.77%
Epoch 1, Loss: 0.0478
Epoch 2, Loss: 0.0410
Epoch 3, Loss: 0.0377
Epoch 4, Loss: 0.0353
Epoch 5, Loss: 0.0334
Accuracy on test set: 97.79%
